In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rohah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in c:\users\rohah\anaconda3\envs\jarvis\lib\site-packages (1.7.11)


In [3]:
import io
from googleapiclient.http import MediaIoBaseDownload
DRIVE = discovery.build('drive', 'v3', http=creds.authorize(Http()))
# if you get the shareable link, the link contains this id, replace the file_id below
file_id = '1Vf8aByqXQwTVmxxs5jKenvAXaxSF6TsH'
request = DRIVE.files().get_media(fileId=file_id)
# replace the filename and extension in the first field below
fh = io.FileIO('wiki-news-300d-1M.vec', mode='w')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))

NameError: name 'discovery' is not defined

In [5]:
from googleapiclient import discovery
from httplib2 import Http
import oauth2client
from oauth2client import file, client, tools
obj = lambda: None
lmao = {"auth_host_name":'localhost', 'noauth_local_webserver':'store_true', 'auth_host_port':[8080, 8090], 'logging_level':'ERROR'}
for k, v in lmao.items():
    setattr(obj, k, v)
    
# authorization boilerplate code
SCOPES = 'https://www.googleapis.com/auth/drive.readonly'
store = file.Storage('token.json')
creds = store.get()
# The following will give you a link if token.json does not exist, the link allows the user to give this app permission
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_id.json', SCOPES)
    creds = tools.run_flow(flow, store, obj)

In [123]:
import numpy as np
import random as rand
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import gensim  
import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split, StratifiedKFold

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs

sns.set_style("whitegrid")
np.random.seed(0)

MAX_NB_WORDS = 100000
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

In [7]:
embeddings_index = {}
f = codecs.open('./wiki-news-300d-1M.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('found %s word vectors' % len(embeddings_index))

232101it [00:29, 7800.70it/s]


found 232101 word vectors


In [8]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)

In [9]:
def splitIntoStem(message):
    return [removeNumeric(stripEmoji(singleCharacterRemove(removePunctuation
                                                           (removeHyperlinks
                                                            (removeHashtags
                                                             (removeUsernames
                                                              (stemWord(word)))))))) for word in message.split()]
def stemWord(tweet):
    return tweet.lower()

def removeUsernames(tweet):
    return re.sub('@[^\s]+', '', tweet)

def removeHashtags(tweet):
    return re.sub(r'#[^\s]+', '', tweet)

def removeHyperlinks(tweet):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet)

def removeNumeric(value):
    blist2 = [item for item in value if not item.isdigit()]
    blist3 = "".join(blist2)
    return blist3

def removePunctuation(tweet):

    return re.sub(r'[^\w\s]','',tweet)

def singleCharacterRemove(tweet):
    return re.sub(r'(?:^| )\w(?:$| )', ' ', tweet)

def stripEmoji(text):

    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    return RE_EMOJI.sub(r'', text)

In [10]:
tweets_df = pd.read_csv('./tweets.csv', sep=',', header=0)
tweet_list = tweets_df['tweet'].tolist()
# label_list = tweets_df['Label'].tolist()
# one_hot_labels = keras.utils.to_categorical(label_list, num_classes=4)
label_names = ["business", "entertainment", "health", "politics"]
labels = tweets_df[label_names].values

In [11]:
processed_list = []
for sentence in tqdm(tweet_list):
    tokens = " ".join(splitIntoStem(sentence)).split()
    filtered = [word for word in tokens if word not in stop_words]
    processed_list.append(" ".join(filtered))

100%|████████████████████████████████████████████████████████████████████████████| 1096/1096 [00:00<00:00, 8791.56it/s]


In [12]:
df = pd.DataFrame()
df["tweet"] = processed_list
df['doc_len'] = df['tweet'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(df['doc_len'].mean() + df['doc_len'].std()).astype(int)
max_seq_len

12

In [13]:
# x_train, x_test, y_train, y_test = train_test_split(processed_list, one_hot_labels, test_size=0.35)
x_train, x_test, y_train, y_test = train_test_split(processed_list, labels, test_size=0.35)

In [14]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train + x_test)
word_seq_train = tokenizer.texts_to_sequences(x_train)
word_seq_test = tokenizer.texts_to_sequences(x_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

dictionary size:  4132


In [15]:
num_tokens = [len(tokens) for tokens in word_seq_train + word_seq_test]
num_tokens = np.array(num_tokens)
num_tokens

array([ 8,  9,  7, ..., 16,  8,  5])

In [16]:
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

In [17]:
word_seq_test = word_seq_test[:-1]
y_test = y_test[:-1]

In [18]:
#training params
batch_size = 256
num_epochs = 8 

#model parameters
num_filters = 64 
embed_dim = 300 
weight_decay = 1e-4

In [19]:
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_dim))
embedding_matrix2 = np.zeros((nb_words, embed_dim))

In [20]:
words_not_found = []

for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print("sample words not found: ", np.random.choice(words_not_found, 10))

number of null word embeddings: 601
sample words not found:  ['modis' 'ptsd' 'brzezinski' 'infowars' 'aamir' 'nyu' 'humantohuman'
 'assange' 'johansson' 'baumol']


In [21]:
for word, i in word_index.items():
    if i >= nb_words:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix2[i] = embedding_vector
    except KeyError:
        embedding_matrix2[i]=np.random.normal(0,np.sqrt(0.25),embed_dim)

del(word_vectors)

In [ ]:
model = Sequential()
model.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
model.add(Dropout(0.2))
model.add(Conv1D(num_filters, 5, padding='same', activation='relu', strides=1))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 5, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model2 = Sequential()
model2.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix2], input_length=max_seq_len, trainable=True))
model2.add(Dropout(0.2))
model2.add(Conv1D(num_filters, 5, padding='same', activation='relu', strides=1))
model2.add(MaxPooling1D(2))
model2.add(Conv1D(num_filters, 5, activation='relu', padding='same'))
model2.add(GlobalMaxPooling1D())
model2.add(Dropout(0.5))
model2.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model2.add(Dense(4, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

In [ ]:
history = model.fit(word_seq_train, y_train, batch_size = batch_size, epochs = num_epochs, shuffle = True, validation_data = (word_seq_test, y_test))

In [ ]:
history2 = model2.fit(word_seq_train, y_train, batch_size = batch_size, epochs = num_epochs, shuffle = True, validation_data = (word_seq_test, y_test))

In [ ]:
model.evaluate(word_seq_test, y_test)

In [ ]:
model2.evaluate(word_seq_test, y_test)

In [ ]:
loss1 = history.history['loss']
val_loss1 = history.history['val_loss']
acc1 = history.history['acc']
val_acc1 = history.history['val_acc']
loss2 = history2.history['loss']
val_loss2 = history2.history['val_loss']
acc2 = history2.history['acc']
val_acc2 = history2.history['val_acc']

epochs = range(1, len(loss1) + 1)

In [ ]:
plt.plot(epochs, loss1, color='red', label='Training loss')
plt.plot(epochs, val_loss1, color='green', label='Validation loss')
plt.title('Training and validation loss (fasttext)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, acc1, color='red', label='Training acc')
plt.plot(epochs, val_acc1, color='green', label='Validation acc')
plt.title('Training and validation accuracy (fasttext)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, loss2, color='red', label='Training loss')
plt.plot(epochs, val_loss2, color='green', label='Validation loss')
plt.title('Training and validation loss (skipgram)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, acc2, color='red', label='Training acc')
plt.plot(epochs, val_acc2, color='green', label='Validation acc')
plt.title('Training and validation accuracy (skipgram)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, acc1, color='red', label='Fasttext acc')
plt.plot(epochs, acc2, color='green', label='Skipgram acc')
plt.title('Training Accuracy fasttext & skipgram')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, val_acc1, color='red', label='Fasttext val_acc')
plt.plot(epochs, val_acc2, color='green', label='Skipgram val_acc')
plt.title('Validating Accuracy fasttext & skipgram')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, loss1, color='red', label='Fasttext loss')
plt.plot(epochs, loss2, color='green', label='Skipgram loss')
plt.title('Training Loss fasttext & skipgram')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, val_loss1, color='red', label='Fasttext val_loss')
plt.plot(epochs, val_loss2, color='green', label='Skipgram val_loss')
plt.title('Validating Loss fasttext & skipgram')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
model3 = Sequential()
model3.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix2], input_length=max_seq_len, trainable=True))
model3.add(Dropout(0.2))
model3.add(Conv1D(num_filters, 5, padding='same', activation='relu', strides=1))
model3.add(MaxPooling1D(2))
model3.add(Conv1D(num_filters, 5, activation='relu', padding='same'))
model3.add(MaxPooling1D(2))
model3.add(Conv1D(num_filters, 5, activation='relu', padding='same'))
model3.add(GlobalMaxPooling1D())
model3.add(Dropout(0.5))
model3.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model3.add(Dense(4, activation='softmax'))
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model3.summary()

In [ ]:
history3 = model3.fit(word_seq_train, y_train, batch_size = batch_size, epochs = num_epochs, shuffle = True, validation_data = (word_seq_test, y_test))

In [ ]:
model3.evaluate(word_seq_test, y_test)

In [ ]:
loss3 = history3.history['loss']
val_loss3 = history3.history['val_loss']
acc3 = history3.history['acc']
val_acc3 = history3.history['val_acc']

In [ ]:
plt.plot(epochs, acc2, color='red', label='Skipgram 2-CNN acc')
plt.plot(epochs, acc3, color='green', label='Skipgram 3-CNN acc')
plt.title('Training Accuracy skipgram CNN(2 Layers, 3 Layers)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(epochs, val_acc2, color='red', label='Skipgram CNN-2 val_acc')
plt.plot(epochs, val_acc3, color='green', label='Skipgram CNN-3 val_acc')
plt.title('Validating Accuracy skipgram CNN(2 Layers, 3 Layers)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
tweets_df2 = pd.read_csv('./tweets2.csv', sep=',', header=0)
tweet_list2 = tweets_df2['tweet']
label_list2 = tweets_df2['labels']

In [ ]:
le = LabelEncoder()
le.fit_transform(label_list2)

In [ ]:
# folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=1).split(word_seq_train, y_train)
skf = StratifiedKFold(n_splits = 10)
skf.get_n_splits(tweet_list2, label_list2)
StratifiedKFold(n_splits=10, random_state=None, shuffle=False)
for train_index, test_index in skf.split(tweet_list2, label_list2):
    X_train, X_test = tweet_list2[train_index], tweet_list2[test_index]
    Y_train, Y_test = label_list2[train_index], label_list2[test_index]

In [ ]:
Y_train

In [ ]:
processed_list = []
for sentence in tqdm(tweet_list2):
    tokens = " ".join(splitIntoStem(sentence)).split()
    filtered = [word for word in tokens if word not in stop_words]
    processed_list.append(" ".join(filtered))

In [ ]:
df = pd.DataFrame()
df["tweet"] = processed_list
df['doc_len'] = df['tweet'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(df['doc_len'].mean() + df['doc_len'].std()).astype(int)
max_seq_len

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train + X_test)
word_seq_train2 = tokenizer.texts_to_sequences(X_train)
word_seq_test2 = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

In [112]:
#Binary to Real Mapping
Num_filters = [32, 64, 128, 256]
Architecture = [2, 3, 2, 2]
Filter_size = [5, 7, 9, 11]
# x -> np array BPSO Particle
x = np.random.randint(2, size = 6)
print(x)
nf = int("".join(map(str, x[0:2])), 2)
ar = int("".join(map(str, x[2:4])), 2)
fs = int("".join(map(str, x[4:6])), 2)
num_filters = Num_filters[nf]
architecture = Architecture[ar]
filter_size = Filter_size[fs]

[0 0 1 1 1 1]


In [113]:
def model(num_filters, architecture, filter_size, word_seq_train, y_train, word_seq_test, y_val):
    model = Sequential()
    model.add(Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
    model.add(Dropout(0.2))
    for i in range(architecture - 1):
        model.add(Conv1D(num_filters, filter_size, padding='same', activation='relu', strides=1))
        model.add(MaxPooling1D(2))
    model.add(Conv1D(num_filters, filter_size, activation='relu', padding='same'))
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    history = model.fit(word_seq_train, y_train, batch_size = batch_size, epochs = num_epochs, shuffle = True, validation_data = (word_seq_test, y_test))
    score, acc = model.evaluate(word_seq_test, y_test)
    print('Val accuracy:', acc)
    return -acc

In [114]:
def cnn_optimizer(x):
    nf = int("".join(map(str, x[0:2])), 2)
    ar = int("".join(map(str, x[2:4])), 2)
    fs = int("".join(map(str, x[4:6])), 2)
    num_filters = Num_filters[nf]
    architecture = Architecture[ar]
    filter_size = Filter_size[fs]
    return model(num_filters, architecture, filter_size, word_seq_train, y_train, word_seq_test, y_test)

In [126]:
# Class, which describes a single particle of a swarm.
class Particle:
    # D - dimension of a search space;
    def __init__(self, D):
        self.velocity = np.zeros(D)
        self.coordinates = np.random.randint(2, size = D)
        self.best_position = np.copy(self.coordinates)
        self.best_score = float('inf') #fuck

    # Update velocity based on particle's and group's best positions.
    def __update_velocity(self, group_best_pos, w, c1, c2):
        self.velocity = w * self.velocity + c1 * rand.random() * (self.best_position - self.coordinates) + c2 * rand.random() * (group_best_pos - self.coordinates)
    
    def compare_update(self, s):
        if s < np.random.rand(1):
            return 1
        else:
            return 0
    
    def sigmoid(x):
        return 1.0 / (1.0 + np.exp(-(x)))
    
    # Update position of a particle.
    def move(self, group_best_pos, w, c1, c2, fitness_function):
        self.__update_velocity(group_best_pos, w, c1, c2)
        self.coordinates = list(map(self.compare_update, list(map(self.sigmoid, self.velocity))))
        current_score = fitness_function(self.coordinates)
    # Evaluate Score
    # Fitness Function to update current_score
        
        if current_score < self.best_score:
            self.best_position = np.copy(self.coordinates)
            self.best_score = current_score

    def sigmoid(self, x):
        return 1.0 / (1.0 + np.exp(-(x)))
    
    def __str__(self):
        return (
        'Current velocity = {0};\nPosition: {1};\nBest position so far: {2};\nScore for the best position = {3}.\n' \
        .format(self.velocity, self.coordinates, self.best_position, self.best_score))

In [127]:
D = 6
N = 6
w = 0.7
c1 = np.random.rand()
c2 = np.random.rand()
iter_num = 5

In [128]:
# Class, which describes our model, it's states of the optimization process.
# Also contains methods for visualizing how the algorithm works.
class Swarm:
    # D - dimension of the search space;
    # N - number of particles to generate;
    # w, c1, c2 - model parameters;
    # fitness_function - function of vector of parameters X to optimize;
    # iter_num - maximum number of iterations.
    def __init__(self, D, N, w, c1, c2, iter_num):
        self.particles = [Particle(D) for _ in range(N)]
        self.dimension = D
        self.particles_num = N
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.iter_num = iter_num

        # Setting random particle as group's initial best position
        rnd_particle = self.particles[np.random.randint(N)]
        self.group_best_position = np.copy(rnd_particle.coordinates)
        self.group_best_score = self.fitness_function(rnd_particle.coordinates)
        for p in self.particles:
            print(p.__str__())

    # Updates best group position and score (is called at every iteration).
    def _update_group_best(self):
        for p in self.particles:
            if p.best_score < self.group_best_score:
                self.group_best_score = p.best_score
                self.group_best_position = np.copy(p.coordinates)

    # Updates the positions of all the particles in the swarm.
    def _move_all(self):
        for p in self.particles:
            p.move(self.group_best_position, self.w, self.c1, self.c2, self.fitness_function)

    # Main loop of the algorithm.
    def optimize(self):
        i = 0
        self._update_group_best()
        while(i < self.iter_num):
            self._move_all()
            self._update_group_best()
            i += 1
            print(i," iteration grp_best: ",self.group_best_position)
            for p in self.particles:
                print(p.__str__())
        return self.group_best_position
    
    def fitness_function(self, x):
        return cnn_optimizer(x)
    
    def __str__(self):
        return 'Best one:{0}'.format(self.group_best_position)

In [129]:
s = Swarm(D, N = N, w = w, c1 = c1, c2 = c2, iter_num = iter_num)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 12, 300)           1239600   
_________________________________________________________________
dropout_38 (Dropout)         (None, 12, 300)           0         
_________________________________________________________________
conv1d_55 (Conv1D)           (None, 12, 32)            67232     
_________________________________________________________________
max_pooling1d_38 (MaxPooling (None, 6, 32)             0         
_________________________________________________________________
conv1d_56 (Conv1D)           (None, 6, 32)             7200      
_________________________________________________________________
max_pooling1d_39 (MaxPooling (None, 3, 32)             0         
_________________________________________________________________
conv1d_57 (Conv1D)           (None, 3, 32)             7200      
__________

In [130]:
s.optimize()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 12, 300)           1239600   
_________________________________________________________________
dropout_40 (Dropout)         (None, 12, 300)           0         
_________________________________________________________________
conv1d_58 (Conv1D)           (None, 12, 64)            96064     
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 6, 64)             0         
_________________________________________________________________
conv1d_59 (Conv1D)           (None, 6, 64)             20544     
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 3, 64)             0         
_________________________________________________________________
conv1d_60 (Conv1D)           (None, 3, 64)             20544     
__________

_________________________________________________________________
dense_42 (Dense)             (None, 4)                 132       
Total params: 1,498,196
Trainable params: 258,596
Non-trainable params: 1,239,600
_________________________________________________________________
Train on 712 samples, validate on 383 samples
Epoch 1/8
712/712 [==============================] - ETA: 3s - loss: 1.4133 - acc: 0.285 - ETA: 0s - loss: 1.4061 - acc: 0.265 - 3s 5ms/step - loss: 1.3982 - acc: 0.2795 - val_loss: 1.3676 - val_acc: 0.4204
Epoch 2/8
712/712 [==============================] - ETA: 0s - loss: 1.3545 - acc: 0.386 - ETA: 0s - loss: 1.3535 - acc: 0.369 - 1s 1ms/step - loss: 1.3515 - acc: 0.3806 - val_loss: 1.3324 - val_acc: 0.4830
Epoch 3/8
712/712 [==============================] - ETA: 0s - loss: 1.3323 - acc: 0.437 - ETA: 0s - loss: 1.3137 - acc: 0.482 - 1s 1ms/step - loss: 1.3059 - acc: 0.4817 - val_loss: 1.2665 - val_acc: 0.5744
Epoch 4/8
712/712 [==============================] - 

Epoch 6/8
712/712 [==============================] - ETA: 0s - loss: 0.6572 - acc: 0.796 - ETA: 0s - loss: 0.6443 - acc: 0.804 - 1s 854us/step - loss: 0.6393 - acc: 0.8020 - val_loss: 0.5597 - val_acc: 0.7990
Epoch 7/8
712/712 [==============================] - ETA: 0s - loss: 0.5740 - acc: 0.769 - ETA: 0s - loss: 0.5267 - acc: 0.802 - 1s 846us/step - loss: 0.5339 - acc: 0.8048 - val_loss: 0.4959 - val_acc: 0.8198
Epoch 8/8
383/383 [==============================] - ETA:  - ETA:  - ETA:  - 0s 457us/step
Val accuracy: 0.8355091385368267
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 12, 300)           1239600   
_________________________________________________________________
dropout_50 (Dropout)         (None, 12, 300)           0         
_________________________________________________________________
conv1d_69 (Conv1D)           (None, 12, 256)           537856 

712/712 [==============================] - ETA: 0s - loss: 0.6607 - acc: 0.773 - ETA: 0s - loss: 0.6231 - acc: 0.793 - 1s 1ms/step - loss: 0.5894 - acc: 0.8048 - val_loss: 0.5598 - val_acc: 0.8120
Epoch 8/8
383/383 [==============================] - ETA:  - ETA:  - ETA:  - 0s 479us/step
Val accuracy: 0.8355091362024412
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, 12, 300)           1239600   
_________________________________________________________________
dropout_54 (Dropout)         (None, 12, 300)           0         
_________________________________________________________________
conv1d_73 (Conv1D)           (None, 12, 32)            48032     
_________________________________________________________________
max_pooling1d_48 (MaxPooling (None, 6, 32)             0         
_________________________________________________________________
conv1d_74 (Conv1D) 

_________________________________________________________________
conv1d_79 (Conv1D)           (None, 6, 256)            721152    
_________________________________________________________________
max_pooling1d_52 (MaxPooling (None, 3, 256)            0         
_________________________________________________________________
conv1d_80 (Conv1D)           (None, 3, 256)            721152    
_________________________________________________________________
global_max_pooling1d_28 (Glo (None, 256)               0         
_________________________________________________________________
dropout_59 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_55 (Dense)             (None, 32)                8224      
_________________________________________________________________
dense_56 (Dense)             (None, 4)                 132       
Total params: 3,535,316
Trainable params: 2,295,716
Non-trainable params: 1,

...

712/712 [==============================] - ETA: 7s - loss: 1.3996 - acc: 0.277 - ETA: 1s - loss: 1.3946 - acc: 0.283 - 6s 9ms/step - loss: 1.3891 - acc: 0.3034 - val_loss: 1.3761 - val_acc: 0.3760
Epoch 2/8
712/712 [==============================] - ETA: 0s - loss: 1.3687 - acc: 0.324 - ETA: 0s - loss: 1.3621 - acc: 0.328 - 2s 2ms/step - loss: 1.3681 - acc: 0.3132 - val_loss: 1.3366 - val_acc: 0.3159
Epoch 3/8
712/712 [==============================] - ETA: 0s - loss: 1.3185 - acc: 0.398 - ETA: 0s - loss: 1.3236 - acc: 0.349 - 2s 2ms/step - loss: 1.3157 - acc: 0.3624 - val_loss: 1.2412 - val_acc: 0.5379
Epoch 4/8
712/712 [==============================] - ETA: 0s - loss: 1.2433 - acc: 0.476 - ETA: 0s - loss: 1.2274 - acc: 0.470 - 2s 3ms/step - loss: 1.2122 - acc: 0.4789 - val_loss: 1.0794 - val_acc: 0.6580
Epoch 5/8
712/712 [==============================] - ETA: 0s - loss: 1.0809 - acc: 0.562 - ETA: 0s - loss: 1.0596 - acc: 0.580 - 2s 3ms/step - loss: 1.0319 - acc: 0.6025 - val_loss: 

712/712 [==============================] - ETA: 6s - loss: 1.4441 - acc: 0.195 - ETA: 1s - loss: 1.4185 - acc: 0.228 - 5s 7ms/step - loss: 1.4081 - acc: 0.2472 - val_loss: 1.3690 - val_acc: 0.3838
Epoch 2/8
712/712 [==============================] - ETA: 0s - loss: 1.3719 - acc: 0.300 - ETA: 0s - loss: 1.3642 - acc: 0.310 - 0s 339us/step - loss: 1.3608 - acc: 0.3216 - val_loss: 1.3513 - val_acc: 0.4439
Epoch 3/8
712/712 [==============================] - ETA: 0s - loss: 1.3506 - acc: 0.332 - ETA: 0s - loss: 1.3381 - acc: 0.335 - 0s 326us/step - loss: 1.3406 - acc: 0.3385 - val_loss: 1.3266 - val_acc: 0.4621
Epoch 4/8
712/712 [==============================] - ETA: 0s - loss: 1.3223 - acc: 0.386 - ETA: 0s - loss: 1.3187 - acc: 0.382 - 0s 328us/step - loss: 1.3138 - acc: 0.3820 - val_loss: 1.2837 - val_acc: 0.4752
Epoch 5/8
712/712 [==============================] - ETA: 0s - loss: 1.2797 - acc: 0.398 - ETA: 0s - loss: 1.2674 - acc: 0.429 - 0s 346us/step - loss: 1.2647 - acc: 0.4368 - va

383/383 [==============================] - ETA:  - ETA:  - 0s 194us/step
Val accuracy: 0.6866840735739267
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_49 (Embedding)     (None, 12, 300)           1239600   
_________________________________________________________________
dropout_94 (Dropout)         (None, 12, 300)           0         
_________________________________________________________________
conv1d_120 (Conv1D)          (None, 12, 128)           268928    
_________________________________________________________________
max_pooling1d_75 (MaxPooling (None, 6, 128)            0         
_________________________________________________________________
conv1d_121 (Conv1D)          (None, 6, 128)            114816    
_________________________________________________________________
global_max_pooling1d_46 (Glo (None, 128)               0         
____________________________________

dense_95 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_96 (Dense)             (None, 4)                 132       
Total params: 1,358,420
Trainable params: 118,820
Non-trainable params: 1,239,600
_________________________________________________________________
Train on 712 samples, validate on 383 samples
Epoch 1/8
712/712 [==============================] - ETA: 8s - loss: 1.3820 - acc: 0.324 - ETA: 2s - loss: 1.3833 - acc: 0.314 - 6s 9ms/step - loss: 1.3795 - acc: 0.3272 - val_loss: 1.3673 - val_acc: 0.3629
Epoch 2/8
712/712 [==============================] - ETA: 0s - loss: 1.3659 - acc: 0.320 - ETA: 0s - loss: 1.3634 - acc: 0.320 - 0s 346us/step - loss: 1.3592 - acc: 0.3230 - val_loss: 1.3458 - val_acc: 0.4151
Epoch 3/8
712/712 [==============================] - ETA: 0s - loss: 1.3543 - acc: 0.300 - ETA: 0s - loss: 1.3313 - acc: 0.347 - 0s 329us/step - loss: 1.3350 - acc: 0.3455 - val_loss: 1.3211 -

array([1, 1, 1, 1, 1, 0])